# Data Description



| Attribute | Description |
|----------|----------|
|Name	|The brand and model of the car
|Location	|The location in which the car is being sold or is available for purchase
|Year	|The year or edition of the model
|Kilometers_Driven	|The total kilometers driven in the car by the previous owner(s) in KM
|Fuel_Type	|The type of fuel used by the car
|Transmission	|The type of transmission used by the car
|Owner_Type	|Whether the ownership is Firsthand, Second hand or other
|Mileage	|The standard mileage offered by the car company in kmpl or km/kg
|Engine	|The displacement volume of the engine in cc
|Power	|The maximum power of the engine in bhp
|Seats	|The number of seats in the car
|New_Price	|Price of new model
|Price	|The price of the used car in INR Lakhs

> Note

        1 INR Lakhs = 100,000 Indian Rupees ~= 1,200 USD

# Importing Libraries

In [1]:
import plotly.figure_factory as ff #1 
import numpy as np  #2
import pandas as pd #3
import pandas_profiling as pp#4
from math import ceil#5
import plotly.express as px #6
import plotly.graph_objects as go #7
import matplotlib as mpl #8
import matplotlib.pyplot as plt  #9
from datasist.structdata import detect_outliers #10
from geopy.distance import great_circle #11
import seaborn as sns #12 
from sklearn.model_selection import train_test_split #13
from sklearn.preprocessing import OneHotEncoder #14
from plotly.subplots import make_subplots  #15
from scipy import stats #16
from datetime import datetime#17
from collections import Counter  #18
from sklearn.impute import SimpleImputer# 19
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, OrdinalEncoder # 20 
from sklearn.linear_model import LinearRegression #21
from sklearn.metrics import r2_score  , mean_squared_error #22

pd.set_option('mode.chained_assignment', None)
sns.set(rc = {'figure.figsize': (12,12)}, font_scale = 1.3)






# Reading Data

In [2]:
df = pd.read_csv('C:/Users/hp/Desktop/Data Science/Lec_25/train.csv', na_values=['N/A', 'no', '?']) 

# Exploring Data

In [3]:
df.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,NaN,1.75
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6019 entries, 0 to 6018
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               6019 non-null   object 
 1   Location           6019 non-null   object 
 2   Year               6019 non-null   int64  
 3   Kilometers_Driven  6019 non-null   int64  
 4   Fuel_Type          6019 non-null   object 
 5   Transmission       6019 non-null   object 
 6   Owner_Type         6019 non-null   object 
 7   Mileage            6017 non-null   object 
 8   Engine             5983 non-null   object 
 9   Power              5983 non-null   object 
 10  Seats              5977 non-null   float64
 11  New_Price          824 non-null    object 
 12  Price              6019 non-null   float64
dtypes: float64(2), int64(2), object(9)
memory usage: 611.4+ KB


In [5]:
df.describe() 

,Year,Kilometers_Driven,Seats,Price
count,6019.000000,6.019000e+03,5977.000000,6019.000000
mean,2013.358199,5.873838e+04,5.278735,9.479468
std,3.269742,9.126884e+04,0.808840,11.187917
min,1998.000000,1.710000e+02,0.000000,0.440000
25%,2011.000000,3.400000e+04,5.000000,3.500000
50%,2014.000000,5.300000e+04,5.000000,5.640000
75%,2016.000000,7.300000e+04,5.000000,9.950000
max,2019.000000,6.500000e+06,10.000000,160.000000


        * Year Range : 1998 - 2019
        * Kilometers_Driven Range : 171 - 650000
        * Seats Range : 0 - 10  (0 is not logical)
        * Price Range : 0.44 - 160

In [6]:
df[df.Seats == 0]       

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
3999,Audi A4 3.2 FSI Tiptronic Quattro,Hyderabad,2012,125000,Petrol,Automatic,First,10.5 kmpl,3197 CC,null bhp,0.0,NaN,18.0


In [7]:
df.Seats = df.Seats.replace(0, np.nan)  

In [8]:
df.Seats.value_counts().sort_values(ascending=False)  

5.0     5014
7.0      674
8.0      134
4.0       99
6.0       31
2.0       16
10.0       5
9.0        3
Name: Seats, dtype: int64

In [9]:
df.describe(include=['O'])   

,Name,Location,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,New_Price
count,6019,6019,6019,6019,6019,6017,5983,5983,824
unique,1876,11,5,2,4,442,146,372,540
top,Mahindra XUV500 W8 2WD,Mumbai,Diesel,Manual,First,18.9 kmpl,1197 CC,74 bhp,95.13 Lakh
freq,49,790,3205,4299,4929,172,606,235,6


In [10]:
# Scatter Matrix
fig = px.scatter_matrix(df, dimensions=['Year', 'Kilometers_Driven', 'Mileage', 'Engine', 'Power', 'Seats', 'Price'])
fig.update_layout(width=1500, height=1200, showlegend=False, title_text="Pairplot of Car Price Prediction Dataset")
fig.show() 


In [11]:
df.Name.str.split().str[0].nunique()  ## Brand names   

31

In [12]:
df.Name.str.contains('Hyundai').sum() 

1107

In [13]:
df.Name.str.split().str[1].nunique()  ## Model names

212

In [14]:
df.Location.value_counts() 

Mumbai        790
Hyderabad     742
Kochi         651
Coimbatore    636
Pune          622
Delhi         554
Kolkata       535
Chennai       494
Jaipur        413
Bangalore     358
Ahmedabad     224
Name: Location, dtype: int64

In [15]:
df.Fuel_Type.value_counts() 

Diesel      3205
Petrol      2746
CNG           56
LPG           10
Electric       2
Name: Fuel_Type, dtype: int64

> Note

        Take care of Electrical cars (only 2) in splitting data

In [16]:
df.Transmission.value_counts()  

Manual       4299
Automatic    1720
Name: Transmission, dtype: int64

In [17]:
df.Owner_Type.value_counts()

First             4929
Second             968
Third              113
Fourth & Above       9
Name: Owner_Type, dtype: int64

# Check Missing Values


In [18]:
df.isnull().sum()

Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 36
Power                  36
Seats                  43
New_Price            5195
Price                   0
dtype: int64

In [19]:
df.isnull().mean() * 100

Name                  0.000000
Location              0.000000
Year                  0.000000
Kilometers_Driven     0.000000
Fuel_Type             0.000000
Transmission          0.000000
Owner_Type            0.000000
Mileage               0.033228
Engine                0.598106
Power                 0.598106
Seats                 0.714404
New_Price            86.310018
Price                 0.000000
dtype: float64

In [20]:
# Drop New_Price column
df.drop('New_Price', axis=1, inplace=True)

# Some Feature Engineering


In [21]:
# Fixing Numerical columns
def fix_numerical(df, col):
    df[col] = pd.to_numeric(df[col].str.split().str[0], errors='coerce')
    return df[col] 

In [22]:
for col in ['Mileage', 'Engine', 'Power']:
    df[col] = fix_numerical(df,col)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6019 entries, 0 to 6018
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               6019 non-null   object 
 1   Location           6019 non-null   object 
 2   Year               6019 non-null   int64  
 3   Kilometers_Driven  6019 non-null   int64  
 4   Fuel_Type          6019 non-null   object 
 5   Transmission       6019 non-null   object 
 6   Owner_Type         6019 non-null   object 
 7   Mileage            6017 non-null   float64
 8   Engine             5983 non-null   float64
 9   Power              5876 non-null   float64
 10  Seats              5976 non-null   float64
 11  Price              6019 non-null   float64
dtypes: float64(5), int64(2), object(5)
memory usage: 564.4+ KB


In [24]:
df.sample(5)

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
2574,Volkswagen Vento Diesel Highline,Mumbai,2011,83000,Diesel,Manual,Second,20.54,1598.0,103.60,5.0,2.90
1566,Hyundai Elantra CRDi SX AT,Mumbai,2013,51000,Diesel,Automatic,First,19.50,1582.0,126.24,5.0,8.75
5468,Tata Nano CX,Kochi,2014,59508,Petrol,Manual,First,25.40,624.0,37.48,4.0,1.26
594,Maruti Swift Dzire VDi,Hyderabad,2011,81759,Diesel,Manual,First,19.30,1248.0,73.90,5.0,4.39
256,Toyota Innova Crysta 2.4 ZX MT,Pune,2017,43000,Diesel,Manual,First,13.68,2393.0,147.80,7.0,18.65


In [25]:
df.describe() 

,Year,Kilometers_Driven,Mileage,Engine,Power,Seats,Price
count,6019.000000,6.019000e+03,6017.000000,5983.000000,5876.000000,5976.000000,6019.000000
mean,2013.358199,5.873838e+04,18.134961,1621.276450,113.253050,5.279618,9.479468
std,3.269742,9.126884e+04,4.582289,601.355233,53.874957,0.806019,11.187917
min,1998.000000,1.710000e+02,0.000000,72.000000,34.200000,2.000000,0.440000
25%,2011.000000,3.400000e+04,15.170000,1198.000000,75.000000,5.000000,3.500000
50%,2014.000000,5.300000e+04,18.150000,1493.000000,97.700000,5.000000,5.640000
75%,2016.000000,7.300000e+04,21.100000,1984.000000,138.100000,5.000000,9.950000
max,2019.000000,6.500000e+06,33.540000,5998.000000,560.000000,10.000000,160.000000


In [26]:
df[df['Mileage'] ==  0]

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
14,Land Rover Freelander 2 TD4 SE,Pune,2012,85000,Diesel,Automatic,Second,0.0,2179.0,115.0,5.0,17.50
67,Mercedes-Benz C-Class Progressive C 220d,Coimbatore,2019,15369,Diesel,Automatic,First,0.0,1950.0,194.0,5.0,35.67
79,Hyundai Santro Xing XL,Hyderabad,2005,87591,Petrol,Manual,First,0.0,1086.0,NaN,5.0,1.30
194,Honda City 1.5 GXI,Ahmedabad,2007,60006,Petrol,Manual,First,0.0,NaN,NaN,NaN,2.95
229,Ford Figo Diesel,Bangalore,2015,70436,Diesel,Manual,First,0.0,1498.0,99.0,NaN,3.60
...,...,...,...,...,...,...,...,...,...,...,...,...
5647,Toyota Qualis Fleet A3,Mumbai,2001,227000,Diesel,Manual,Fourth & Above,0.0,2446.0,NaN,8.0,2.20
5875,Mercedes-Benz C-Class Progressive C 220d,Ahmedabad,2019,4000,Diesel,Automatic,First,0.0,1950.0,194.0,5.0,35.00
5943,Mahindra Jeep MM 540 DP,Chennai,2002,75000,Diesel,Manual,First,0.0,2112.0,NaN,6.0,1.70
5972,Hyundai Santro Xing GL,Mumbai,2008,65000,Petrol,Manual,Second,0.0,1086.0,62.0,5.0,1.39


In [27]:
df.isnull().sum()

Name                   0
Location               0
Year                   0
Kilometers_Driven      0
Fuel_Type              0
Transmission           0
Owner_Type             0
Mileage                2
Engine                36
Power                143
Seats                 43
Price                  0
dtype: int64

In [28]:
#df['Brand'] = df.Name.str.split().str[0]
df['Name'] = df.Name.str.split().str[0:2].str.join(' ')


In [29]:
df['Age'] = df.Year.max() - df.Year
df.drop('Year', axis=1, inplace=True)

In [30]:
df.sample()

,Name,Location,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,Age
2122,Honda City,Delhi,55000,Petrol,Manual,First,17.8,1497.0,117.3,5.0,5.84,5


In [31]:
df.corr()

,Kilometers_Driven,Mileage,Engine,Power,Seats,Price,Age
Kilometers_Driven,1.000000,-0.065253,0.091068,0.033503,0.084209,-0.011493,0.173048
Mileage,-0.065253,1.000000,-0.597699,-0.537729,-0.311268,-0.306593,-0.321565
Engine,0.091068,-0.597699,1.000000,0.866185,0.397848,0.658354,0.052197
Power,0.033503,-0.537729,0.866185,1.000000,0.101562,0.772566,-0.014525
Seats,0.084209,-0.311268,0.397848,0.101562,1.000000,0.053247,-0.011909
Price,-0.011493,-0.306593,0.658354,0.772566,0.053247,1.000000,-0.305327
Age,0.173048,-0.321565,0.052197,-0.014525,-0.011909,-0.305327,1.000000


In [32]:
# Heatmap
px.imshow(df.corr(), width=800, height=800, title="Heatmap of Car Price Prediction Dataset")



In [33]:
px.scatter(df, x='Engine', y='Price', trendline='ols', width=800, height=600, title="Scatterplot of Engine vs Price")

In [34]:
px.scatter(df, x='Power', y='Price', trendline='ols', width=800, height=600, title= "Scatterplot of Power vs Price")

# Data Splitting

In [35]:
# Data Splitting into features and target
X = df.drop('Price', axis=1)
y = df['Price']

# Splitting into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=X['Fuel_Type'])

In [36]:
X_train.Fuel_Type.value_counts()

Diesel      2564
Petrol      2197
CNG           45
LPG            8
Electric       1
Name: Fuel_Type, dtype: int64

# Data Preprocessing

In [37]:
numerical_cols = list(df.select_dtypes(include=['int64', 'float64']).columns)
numerical_cols.remove('Price')
numerical_cols


['Kilometers_Driven', 'Mileage', 'Engine', 'Power', 'Seats', 'Age']

In [38]:
categorical_cols = list(df.select_dtypes(include=['object']).columns)
categorical_cols

['Name', 'Location', 'Fuel_Type', 'Transmission', 'Owner_Type']

## Handling Missing Values

In [39]:
df.isnull().sum()

Name                   0
Location               0
Kilometers_Driven      0
Fuel_Type              0
Transmission           0
Owner_Type             0
Mileage                2
Engine                36
Power                143
Seats                 43
Price                  0
Age                    0
dtype: int64

In [40]:
missing_cols = ['Mileage', 'Engine', 'Power', 'Seats']
for col in missing_cols:
    fig = px.histogram(df, x=col, width=800, height=400)
    fig.show()

In [41]:
# Impute missing values using SimpleImputer

imputer = SimpleImputer(strategy='median')
X_train[missing_cols] = imputer.fit_transform(X_train[missing_cols])
X_test[missing_cols] = imputer.transform(X_test[missing_cols])



In [42]:
df.isnull().sum()

Name                   0
Location               0
Kilometers_Driven      0
Fuel_Type              0
Transmission           0
Owner_Type             0
Mileage                2
Engine                36
Power                143
Seats                 43
Price                  0
Age                    0
dtype: int64

In [43]:
X_train.isnull().sum()

Name                 0
Location             0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
Mileage              0
Engine               0
Power                0
Seats                0
Age                  0
dtype: int64

In [44]:
X_test.isnull().sum()

Name                 0
Location             0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
Mileage              0
Engine               0
Power                0
Seats                0
Age                  0
dtype: int64

# Scaling Data

In [45]:
X_train.describe() 

,Kilometers_Driven,Mileage,Engine,Power,Seats,Age
count,4.815000e+03,4815.000000,4815.000000,4815.000000,4815.000000,4815.000000
mean,5.888102e+04,18.153776,1618.088889,112.321981,5.274143,5.668951
std,9.996964e+04,4.576099,595.994205,52.545529,0.796770,3.290287
min,1.710000e+02,0.000000,72.000000,34.200000,2.000000,0.000000
25%,3.400450e+04,15.260000,1198.000000,78.900000,5.000000,3.000000
50%,5.300000e+04,18.160000,1493.000000,94.000000,5.000000,5.000000
75%,7.300000e+04,21.100000,1968.000000,138.030000,5.000000,8.000000
max,6.500000e+06,33.540000,5998.000000,560.000000,10.000000,21.000000


In [46]:
# Standardize numerical columns

scaler = MinMaxScaler()
#scaler = StandardScaler()
#scaler = RobustScaler()
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

In [47]:
X_train.describe()

,Kilometers_Driven,Mileage,Engine,Power,Seats,Age
count,4815.000000,4815.000000,4815.000000,4815.000000,4815.000000,4815.000000
mean,0.009033,0.541257,0.260899,0.148577,0.409268,0.269950
std,0.015380,0.136437,0.100573,0.099934,0.099596,0.156680
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.005205,0.454979,0.190010,0.085013,0.375000,0.142857
50%,0.008128,0.541443,0.239791,0.113731,0.375000,0.238095
75%,0.011205,0.629100,0.319946,0.197471,0.375000,0.380952
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [48]:
px.scatter(x = X_train['Seats'], y= y_train)

## Handling Categorical Data

In [49]:
df.describe(include=['O'])

,Name,Location,Fuel_Type,Transmission,Owner_Type
count,6019,6019,6019,6019,6019
unique,216,11,5,2,4
top,Maruti Swift,Mumbai,Diesel,Manual,First
freq,353,790,3205,4299,4929


In [50]:
X_train.Owner_Type.value_counts()

First             3930
Second             775
Third              101
Fourth & Above       9
Name: Owner_Type, dtype: int64

In [51]:
# Ordinal Encoding for Owner_Type 

ordinal_encoder = OrdinalEncoder(categories=[['First', 'Second', 'Third', 'Fourth & Above']])
X_train['Owner_Type'] = ordinal_encoder.fit_transform(X_train[['Owner_Type']])
X_test['Owner_Type'] = ordinal_encoder.transform(X_test[['Owner_Type']]) 



In [52]:
X_train['Owner_Type'].value_counts()

0.0    3930
1.0     775
2.0     101
3.0       9
Name: Owner_Type, dtype: int64

In [53]:
X_train.sample(5)  

,Name,Location,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Age
5983,Volkswagen Polo,Kolkata,0.008436,Petrol,Manual,0.0,0.491055,0.190010,0.075504,0.375,0.380952
1808,Toyota Innova,Kochi,0.009537,Diesel,Manual,0.0,0.387299,0.408707,0.125143,0.625,0.285714
4311,Mercedes-Benz E-Class,Pune,0.015359,Diesel,Automatic,0.0,0.327967,0.491900,0.312476,0.375,0.476190
9,Tata Indica,Chennai,0.010117,Diesel,Manual,1.0,0.664878,0.198448,0.075694,0.375,0.333333
261,Hyundai Verna,Hyderabad,0.022017,Diesel,Manual,0.0,0.655933,0.254809,0.174971,0.375,0.523810


In [54]:
X_train.Name.nunique()

206

In [55]:
#X_train.Brand.nunique()

In [56]:
X_train.Location.nunique() 

11

> Note

        * OneHotEncoder is not used because it will create a lot of columns


In [57]:
#pip install category_encoders

In [58]:
# Handling Categorical columns using Binary Encoding

from category_encoders import BinaryEncoder

encoder = BinaryEncoder(cols=[ 'Name', 'Location', 'Fuel_Type', 'Transmission'])
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test) 
                                                     
                                                         

In [59]:
X_train

,Name_0,Name_1,Name_2,Name_3,Name_4,Name_5,Name_6,Name_7,Location_0,Location_1,...,Fuel_Type_1,Fuel_Type_2,Transmission_0,Transmission_1,Owner_Type,Mileage,Engine,Power,Seats,Age
5320,0,0,0,0,0,0,0,1,0,0,...,0,1,0,1,0.0,0.483005,0.190179,0.075694,0.375,0.142857
3406,0,0,0,0,0,0,1,0,0,0,...,1,0,1,0,1.0,0.596303,0.349477,0.139026,0.375,0.238095
2948,0,0,0,0,0,0,1,1,0,0,...,1,0,1,0,0.0,0.574538,0.349477,0.253747,0.375,0.047619
2909,0,0,0,0,0,1,0,0,0,1,...,0,1,0,1,0.0,0.534287,0.171110,0.053062,0.375,0.523810
3599,0,0,0,0,0,1,0,1,0,1,...,0,1,0,1,0.0,0.590638,0.171110,0.064378,0.375,0.523810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3347,0,0,0,1,0,0,1,1,0,0,...,0,1,1,0,0.0,0.551580,0.189841,0.093077,0.375,0.095238
237,0,0,0,0,1,0,0,1,1,0,...,0,1,0,1,0.0,0.518784,0.240466,0.158045,0.375,0.380952
2461,0,1,0,1,1,1,0,1,0,0,...,1,0,0,1,1.0,0.566786,0.234391,0.141308,0.375,0.190476
5868,0,1,0,0,0,1,0,0,0,1,...,1,0,1,0,0.0,0.244484,0.492913,0.400913,0.250,0.238095


In [60]:
px.histogram(y_train)

In [61]:
# Log Transforming the target variable
y_train = np.log1p(y_train)
y_test = np.log1p(y_test) 




In [62]:
px.histogram(y_train)

# Model Building

In [63]:
# Linear Regression

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lr = LinearRegression()
lr.fit(X_train, y_train)

print('Training Score: ', lr.score(X_train, y_train))
print('Testing Score: ', lr.score(X_test, y_test))
print('RMSE: ', np.sqrt(mean_squared_error(y_test, lr.predict(X_test))))

Training Score:  0.8730236523985905
Testing Score:  0.894329184971067
RMSE:  0.24810651119202695


In [64]:
prices = np.expm1(lr.predict(X_test))
prices

array([19.29843992,  4.71733547, 38.72555513, ..., 21.91685793,
        4.30258968,  6.08160891])

In [65]:
# Random Forest Regressor

from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=100, random_state=42)

rf.fit(X_train, y_train)

print('Training Score: ', rf.score(X_train, y_train))
print('Testing Score: ', rf.score(X_test, y_test))
print('RMSE: ', np.sqrt(mean_squared_error(y_test, rf.predict(X_test))))

Training Score:  0.9903820747758886
Testing Score:  0.9476813530230542
RMSE:  0.17457774735951792


# Predicting Test Data

In [66]:
df_test = pd.read_csv('test.csv')
df_test.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price
0,Maruti Alto K10 LXI CNG,Delhi,2014,40929,CNG,Manual,First,32.26 km/kg,998 CC,58.2 bhp,4.0,NaN
1,Maruti Alto 800 2016-2019 LXI,Coimbatore,2013,54493,Petrol,Manual,Second,24.7 kmpl,796 CC,47.3 bhp,5.0,NaN
2,Toyota Innova Crysta Touring Sport 2.4 MT,Mumbai,2017,34000,Diesel,Manual,First,13.68 kmpl,2393 CC,147.8 bhp,7.0,25.27 Lakh
3,Toyota Etios Liva GD,Hyderabad,2012,139000,Diesel,Manual,First,23.59 kmpl,1364 CC,null bhp,5.0,NaN
4,Hyundai i20 Magna,Mumbai,2014,29000,Petrol,Manual,First,18.5 kmpl,1197 CC,82.85 bhp,5.0,NaN


In [67]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1234 entries, 0 to 1233
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               1234 non-null   object 
 1   Location           1234 non-null   object 
 2   Year               1234 non-null   int64  
 3   Kilometers_Driven  1234 non-null   int64  
 4   Fuel_Type          1234 non-null   object 
 5   Transmission       1234 non-null   object 
 6   Owner_Type         1234 non-null   object 
 7   Mileage            1234 non-null   object 
 8   Engine             1224 non-null   object 
 9   Power              1224 non-null   object 
 10  Seats              1223 non-null   float64
 11  New_Price          182 non-null    object 
dtypes: float64(1), int64(2), object(9)
memory usage: 115.8+ KB


In [68]:
# Preprocessing Test Data
df_test.drop('New_Price', axis=1, inplace=True)

for col in ['Mileage', 'Engine', 'Power']:

    df_test[col] = fix_numerical(df_test, col)

df_test['Name'] = df_test.Name.str.split().str[0:2].str.join(' ')

df_test['Age'] = df_test.Year.max() - df_test.Year

df_test.drop('Year', axis=1, inplace=True)

df_test[missing_cols] = imputer.transform(df_test[missing_cols])

df_test[numerical_cols] = scaler.transform(df_test[numerical_cols]) 

df_test['Owner_Type'] = ordinal_encoder.transform(df_test[['Owner_Type']])

df_test = encoder.transform(df_test)

df_test.head()

,Name_0,Name_1,Name_2,Name_3,Name_4,Name_5,Name_6,Name_7,Location_0,Location_1,...,Fuel_Type_1,Fuel_Type_2,Transmission_0,Transmission_1,Owner_Type,Mileage,Engine,Power,Seats,Age
0,0,0,1,1,1,1,1,0,1,0,...,1,1,0,1,0.0,0.961837,0.156261,0.045645,0.250,0.238095
1,0,0,1,1,1,1,1,0,0,0,...,0,1,0,1,1.0,0.736434,0.122173,0.024914,0.375,0.285714
2,0,0,0,0,1,1,1,1,0,0,...,1,0,0,1,0.0,0.407871,0.391664,0.216052,0.625,0.095238
3,0,0,1,1,1,0,0,1,0,1,...,1,0,0,1,0.0,0.703339,0.218022,0.113731,0.375,0.333333
4,0,0,1,0,1,0,1,1,0,0,...,0,1,0,1,0.0,0.551580,0.189841,0.092526,0.375,0.238095


In [69]:
# Predicting on Test Data
prices = np.expm1(rf.predict(df_test))
prices


array([ 3.1256644 ,  2.50521205, 17.54129924, ...,  2.54259352,
        5.38031329, 19.42773439])